In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from selenium.webdriver.common.by import By

url='https://www.premierleague.com/'

# get the webpage
webpage = requests.get(url)

# load webpage into bs4
soup = BeautifulSoup(webpage.content, 'html.parser')

In [25]:
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.maximize_window()
driver.get(url)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [C:\Users\kah_chen_chan\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
C:\Users\KAH_CH~1\AppData\Local\Temp/ipykernel_6440/2054763912.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [26]:
time.sleep(6) #wait for page to load

driver.find_element(By.XPATH,"/html/body/div[2]/div/div/div[1]/div[5]/button[1]").click() # accept cookies
time.sleep(2)

if len(driver.find_elements(By.ID, "advertClose"))>0:
    driver.find_element(By.XPATH,"//*[@id='advertClose']").click() # close ads
    time.sleep(1)
else:
    time.sleep(1)
    
driver.find_element(By.XPATH,"/html/body/header/div/nav/ul/li[4]/a").click()
time.sleep(8)

if len(driver.find_elements(By.ID, "advertClose"))>0:
    driver.find_element(By.XPATH,"//*[@id='advertClose']").click() # close ads
    time.sleep(1)
else:
    time.sleep(1)

soup = BeautifulSoup(driver.page_source, 'html.parser') #to rerun after change page

tables = []
team_season = []
team_position = []
team_club = []
team_played = []
team_won = []
team_drawn = []
team_lost = []
team_gf = []
team_ga = []
team_gd = []
team_points = []

tableSeasonSeq = 3

while tableSeasonSeq <= 5: # previous 3 seasons
    
    driver.find_element(By.XPATH,"/html/body/main/div[2]/div[1]/div[1]/section/div[2]/div[2]").click() # open Season
    time.sleep(1)
    driver.find_element(By.XPATH,"/html/body/main/div[2]/div[1]/div[1]/section/div[2]/ul/li[" + str(tableSeasonSeq) + "]").click() # click from 2015/16
    time.sleep(3)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser') #to rerun after change page
    
    tPosition_list = soup.find('tbody', attrs={'class':'tableBodyContainer'}).find_all('td', attrs={'class':'pos button-tooltip'})

    for tPosition_items in tPosition_list:

        for tPosition_items2 in tPosition_items.find('span', attrs={'class':'value'}):

            team_position.append(tPosition_items2.text)
            current_tableSeason = soup.find('div', attrs={'data-dropdown-current':'compSeasons'})
            team_season.append(current_tableSeason.text)


    tClub_list = soup.find('tbody', attrs={'class':'tableBodyContainer'}).find_all('td', attrs={'class':'team'})

    for tClub_items in tClub_list:

        for tClub_items2 in tClub_items.find('span', attrs={'class':'long'}):

            team_club.append(tClub_items2.text)


    tableRowSeq = 1
    while tableRowSeq <= 39:

        played_score = driver.find_element(By.XPATH,"/html/body/main/div[2]/div[1]/div[5]/div/div/div/table/tbody/tr[" + str(tableRowSeq) +"]/td[4]")
        team_played.append(played_score.text)
        won_score = driver.find_element(By.XPATH,"/html/body/main/div[2]/div[1]/div[5]/div/div/div/table/tbody/tr[" + str(tableRowSeq) +"]/td[5]")
        team_won.append(won_score.text)
        drawn_score = driver.find_element(By.XPATH,"/html/body/main/div[2]/div[1]/div[5]/div/div/div/table/tbody/tr[" + str(tableRowSeq) +"]/td[6]")
        team_drawn.append(drawn_score.text)
        lost_score = driver.find_element(By.XPATH,"/html/body/main/div[2]/div[1]/div[5]/div/div/div/table/tbody/tr[" + str(tableRowSeq) +"]/td[7]")
        team_lost.append(lost_score.text)
        gf_score = driver.find_element(By.XPATH,"/html/body/main/div[2]/div[1]/div[5]/div/div/div/table/tbody/tr[" + str(tableRowSeq) +"]/td[8]")
        team_gf.append(gf_score.text)
        ga_score = driver.find_element(By.XPATH,"/html/body/main/div[2]/div[1]/div[5]/div/div/div/table/tbody/tr[" + str(tableRowSeq) +"]/td[9]")
        team_ga.append(ga_score.text)
        gd_score = driver.find_element(By.XPATH,"/html/body/main/div[2]/div[1]/div[5]/div/div/div/table/tbody/tr[" + str(tableRowSeq) +"]/td[10]")
        team_gd.append(gd_score.text)
        points_score = driver.find_element(By.XPATH,"/html/body/main/div[2]/div[1]/div[5]/div/div/div/table/tbody/tr[" + str(tableRowSeq) +"]/td[11]")
        team_points.append(points_score.text)

        tableRowSeq = tableRowSeq+2

        
    tableSeasonSeq = tableSeasonSeq + 1


In [27]:
tables = {'Season' : team_season,
         'Position' : team_position,
         'Club' : team_club,
         'Played' : team_played,
         'Won' : team_won,
         'Drawn' : team_drawn,
         'Lost' : team_lost,
         'Goals For' : team_gf,
         'Goals Against' : team_ga,
         'Goal Difference' : team_gd,
         'Points' : team_points}

In [28]:
pl_table_df = pd.DataFrame(tables)
pl_table_df


,Season,Position,Club,Played,Won,Drawn,Lost,Goals For,Goals Against,Goal Difference,Points
0,2020/21,1,Manchester City,38,27,5,6,83,32,51,86
1,2020/21,2,Manchester United,38,21,11,6,73,44,29,74
2,2020/21,3,Liverpool,38,20,9,9,68,42,26,69
3,2020/21,4,Chelsea,38,19,10,9,58,36,22,67
4,2020/21,5,Leicester City,38,20,6,12,68,50,18,66
5,2020/21,6,West Ham United,38,19,8,11,62,47,15,65
6,2020/21,7,Tottenham Hotspur,38,18,8,12,68,45,23,62
7,2020/21,8,Arsenal,38,18,7,13,55,39,16,61
8,2020/21,9,Leeds United,38,18,5,15,62,54,8,59
9,2020/21,10,Everton,38,17,8,13,47,48,-1,59


In [29]:
pl_table_df.to_csv('core/media/data/premierleague_tables.csv', index=False)

In [ ]:
##driver.close()